## Test and visualize sign detector

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from ast import literal_eval

from tqdm import tqdm

import torch

In [2]:
import matplotlib.pyplot as plt

In [3]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
relative_path = '../../'
# ensure that parent path is on the python path in order to have all packages available
import sys, os
parent_path = os.path.join(os.getcwd(), relative_path)
parent_path = os.path.realpath(parent_path)  # os.path.abspath(...)
sys.path.insert(0, parent_path)

In [5]:
from lib.datasets.cunei_dataset_segments import CuneiformSegments, get_segment_meta
from lib.models.trained_model_loader import get_fpn_ssd_net
from lib.detection.run_gen_ssd_detection import gen_ssd_detections

/home/tobias/.virtualenvs/pytorch/local/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.DeprecatedIn23,


In [6]:
collections = ['test']   # e.g. train test saa06
only_annotated = True
only_assigned = True

# store detections for re-use
save_detections = False

# show detections
show_detections = True

In [7]:
model_version = 'v191ft01' 

arch_type = 'mobile'  # resnet, mobile
arch_opt = 1
width_mult = 0.625  # 0.5 0.625 0.75

crop_shape = [600, 600]
tile_shape = [600, 600]

num_classes = 240

with_64 = False 
create_bg_class = False 
with_4_aspects = False  

<a id='complconf'>Config Completeness</a>

[Jump to results](#results)

In [8]:
test_min_score_thresh = 0.01  # 0.01 0.05
test_nms_thresh = 0.5 

eval_ovthresh = 0.5 # 0.4

In [9]:
### Load Model

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
fpnssd_net = get_fpn_ssd_net(model_version, device, arch_type, with_64, arch_opt, width_mult, 
                             relative_path, num_classes, num_c=1, rnd_init_model=False)

print(fpnssd_net)

FPNSSD(
  (fpn): MobileNetV2FPN(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 20, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
      )
      (1): MobileBlock(
        (mobile_block): Sequential(
          (0): InvertedResidual(
            (conv): Sequential(
              (0): Conv2d(20, 20, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU6(inplace)
              (3): Conv2d(20, 20, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=20, bias=False)
              (4): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (5): ReLU6(inplace)
              (6): Conv2d(20, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (7): BatchNorm2d(10, e

In [12]:
### Test net
loc_preds, cls_preds = fpnssd_net(torch.randn(1, 1, 1024, 1024).to(device))
print(loc_preds.size(), cls_preds.size())

(torch.Size([1, 15360, 4]), torch.Size([1, 15360, 240]))


In [13]:
### Prepare dataset

In [14]:
dataset = CuneiformSegments(collections=collections, relative_path=relative_path, 
                            only_annotated=only_annotated, only_assigned=only_assigned, preload_segments=False)

Setup dataset spanning 3 collections with 4465 annotations [67 segments, 67 indices]


In [15]:
### Predict

In [16]:
# filter collection dataset - OPTIONAL
didx_list = range(len(dataset))
didx_list = didx_list[:19]  #19

### Generate ssd detections
(list_seg_ap, 
 list_seg_name_with_anno) = gen_ssd_detections(didx_list, dataset, collections[0], relative_path, 
                                               model_version, fpnssd_net, with_64, create_bg_class, device,
                                               test_min_score_thresh, test_nms_thresh, eval_ovthresh,
                                               save_detections, show_detections, with_4_aspects=with_4_aspects, 
                                               verbose_mode=True)

train:   5%|▌         | 1/19 [00:00<00:15,  1.20it/s]

('P334921', 'Obv')
mAP 0.7926 | global AP: 0.7473 | mAP (align): 0.8859
total_tp: 22 | total_fp: 17 [46] | acc: 0.56
('P334921', 'Rev')


train:  11%|█         | 2/19 [00:01<00:08,  1.98it/s]

mAP 0.7143 | global AP: 0.7286 | mAP (align): 1.0
total_tp: 7 | total_fp: 1 [9] | acc: 0.88
('P334863', 'Obv')


/home/tobias/Dropbox/NeuralNets/caffe_workspace/pycaffe/cuneiform-sign-detection/lib/evaluations/sign_evaluator.py:184: RuntimeWarning: invalid value encountered in divide
  return num_tp, num_fp, num_fp_global, num_tp / float(num_tp + num_fp)
train:  16%|█▌        | 3/19 [00:01<00:06,  2.43it/s]

...

('P334894', 'Rev')
mAP 1.0 | global AP: 0.8218 | mAP (align): 1.0
total_tp: 15 | total_fp: 9 [39] | acc: 0.62


train:  95%|█████████▍| 18/19 [00:05<00:00,  3.14it/s]

('P334894', 'Obv')
mAP 0.8727 | global AP: 0.8457 | mAP (align): 0.8727
total_tp: 41 | total_fp: 39 [62] | acc: 0.51


train: 100%|██████████| 19/19 [00:06<00:00,  3.14it/s]

('P336178', 'Obv')
mAP 0.9375 | global AP: 0.8319 | mAP (align): 0.9375
total_tp: 31 | total_fp: 20 [34] | acc: 0.61
train | v191ft01
RESULTS ON FULL COLLECTION :
mAP 0.7739 | global AP: 0.7816 | mAP (align): 0.7958
total_tp: 471 | total_fp: 690 [792] | prec: 0.406


<a id='results'>Results</a>

[Jump to completeness config](#complconf)

[Jump to Results](#results)